Assignment 3

Implement from scratch an RBM and apply it to DSET3. The RBM should be implemented fully by you (both CD-1 training and inference steps) but you are free to use library functions for the rest (e.g. image loading and management, etc.).

1.     Train an RBM with a number of hidden neurons selected by you (single layer) on the MNIST data (use the training set split provided by the website).

2.     Use the trained RBM to encode a selection of test images (e.g. using one per digit type) using the corresponding activation of the hidden neurons.

3.    Train a simple classifier (e.g. any simple classifier in scikit) to recognize the MNIST digits using as inputs their encoding obtained at step 2. Use the standard training/test split. Show a performance metric of your choice in the presentation/handout.

In [1]:
import matplotlib.pyplot as plt
import idx2numpy
tr_images=idx2numpy.convert_from_file('./dataset/train-images.idx3-ubyte')
tr_labels=idx2numpy.convert_from_file('./dataset/train-labels.idx1-ubyte')
ts_images=idx2numpy.convert_from_file('./dataset/t10k-images.idx3-ubyte')
ts_labels=idx2numpy.convert_from_file('./dataset/t10k-labels.idx1-ubyte')

In [2]:
import numpy as np
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [3]:
class RBM:
    def __init__(self,visible_size,hidden_size):
        self.visible_neurons= np.array(visible_size,dtype='float64')
        self.hidden_neurons= np.array(hidden_size,dtype='float64')

        self.visible_bias= np.zeros(visible_size,dtype='float64')
        self.hidden_bias= np.zeros(hidden_size,dtype='float64')

        self.weights=np.random.normal(loc=0,scale=0.01,size=(visible_size,hidden_size))
    
    def learn(self,values,eta=0.01):
        # clamp data as input
        clamped_data= (values > np.random.rand(*values.shape)).astype(int)
        #sample h given v
        ha_prob= sigmoid(clamped_data@self.weights+self.hidden_bias)
        ha_states= (ha_prob > np.random.rand(*ha_prob.shape)).astype(int)

        #calculate wake part
        wake=clamped_data.T@ha_prob
        #sample v given h
        recon_prob=sigmoid(ha_states@self.weights.T+self.visible_bias)
        recon_act= (recon_prob > np.random.rand(*recon_prob.shape)).astype(int)
        #calculate probability for dream part
        active_prob=sigmoid(recon_act@self.weights+ self.hidden_bias)
        #calculate dream part
        dream=recon_act.T@active_prob

        # update model parameters
        batch_size=np.float64(len(values))

        delta_w=(wake-dream)/batch_size
        delta_bh=( np.sum(ha_prob) - np.sum(recon_prob) )/batch_size
        delta_bv=( np.sum(clamped_data) - np.sum(recon_act) )/batch_size

        self.weights+=eta*delta_w
        self.hidden_bias+=eta*delta_bh
        self.visible_bias+=eta*delta_bv
        #calculate and return reconstruction error
        return np.sum((clamped_data-recon_act)**2)/batch_size
    def train(self,data,epochs=100,learning_rate=0.01):
        print(f"begin training with {len(data)} elements")
        for i in range(0,epochs):
            rec_error=self.learn(data,learning_rate)
            print(f"epoch no.{i+1}, reconstruction error {rec_error}")
    def encode(self,data):
        #clamped_data= (data > np.random.rand(*data.shape)).astype(np.float64)
        #sample h given v
        ha_prob= sigmoid(data@self.weights+self.hidden_bias)
        ha_states= (ha_prob > np.random.rand(*ha_prob.shape)).astype(np.float64)
        return ha_states

In [4]:
epochs=100
rbm=RBM(28*28,500)
training=tr_images.reshape((-1,28*28))
#binarizing data
training_data=((training-training.min())/(training.max()-training.min()) > 0.5).astype(np.float64)
rbm.train(training_data,20,0.001)

begin training with 60000 elements
epoch no.1, reconstruction error 391.5600166666667
epoch no.2, reconstruction error 343.10215
epoch no.3, reconstruction error 308.3268166666667
epoch no.4, reconstruction error 282.57913333333335
epoch no.5, reconstruction error 263.1804333333333
epoch no.6, reconstruction error 248.08178333333333
epoch no.7, reconstruction error 236.05825
epoch no.8, reconstruction error 226.30575
epoch no.9, reconstruction error 217.99255
epoch no.10, reconstruction error 211.00045
epoch no.11, reconstruction error 205.10296666666667
epoch no.12, reconstruction error 199.61708333333334
epoch no.13, reconstruction error 194.7347
epoch no.14, reconstruction error 190.20548333333332
epoch no.15, reconstruction error 185.84405
epoch no.16, reconstruction error 181.72921666666667
epoch no.17, reconstruction error 177.93091666666666
epoch no.18, reconstruction error 174.26603333333333
epoch no.19, reconstruction error 170.98075
epoch no.20, reconstruction error 167.80456

In [5]:
#number of samples per single digit
digit_samples=100

idx=np.array([ np.where(ts_labels == i)[0][:digit_samples] for i in range(10)]).reshape((-1))
test_data=ts_images.reshape((-1,28*28))

In [6]:
h_training_data=rbm.encode(test_data[idx])
h_test_data=rbm.encode(training)

In [7]:
from sklearn.naive_bayes import GaussianNB

nb=GaussianNB().fit(training,tr_labels)
pred=nb.predict(test_data[idx])
len(pred==ts_labels[idx])/len(idx)


1.0

In [8]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300).fit(training,tr_labels)
pred=mlp.predict(test_data[idx])
len(pred==ts_labels[idx])/len(idx)

1.0

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=9).fit(training, tr_labels)
pred=knn.predict(test_data[idx])
len(pred==ts_labels[idx])/len(idx)

1.0